<a href="https://colab.research.google.com/github/pushkar-khetrapal/RealTimePanoptic-TensorRT/blob/master/RealTimePanoptic_TensorRT_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
#Uninstall the current CUDA version
!sudo apt-get --purge remove cuda nvidia* libnvidia-*
!sudo dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!sudo apt-get remove cuda-*
!sudo apt autoremove

In [ ]:
# Installing CUDA 10.0

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo apt-get update
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt install -y ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt-get update

# Install NVIDIA driver
!sudo apt-get -y installnvidia-driver-418

# Install development and runtime libraries (~4GB)
!sudo apt-get install -y \
    cuda-10-0 \
    libcudnn7=7.6.2.24-1+cuda10.0  \
    libcudnn7-dev=7.6.2.24-1+cuda10.0 --allow-change-held-packages

In [ ]:
!nvcc --version

In [ ]:
!wget 'https://www.dropbox.com/s/1qadpocgzspmb6j/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb?dl=0'
!mv '/content/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb?dl=0' '/content/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb'

In [ ]:
!sudo dpkg -i "/content/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt7.0.0.11-ga-20191216_1-1_amd64.deb"
!sudo apt-key add /var/nv-tensorrt-repo-cuda10.0-trt7.0.0.11-ga-20191216/7fa2af80.pub
!sudo apt-get update

In [ ]:
!sudo apt-get install libnvinfer7=7.0.0-1+cuda10.0 libnvonnxparsers7=7.0.0-1+cuda10.0 libnvparsers7=7.0.0-1+cuda10.0 libnvinfer-plugin7=7.0.0-1+cuda10.0 libnvinfer-dev=7.0.0-1+cuda10.0 libnvonnxparsers-dev=7.0.0-1+cuda10.0 libnvparsers-dev=7.0.0-1+cuda10.0 libnvinfer-plugin-dev=7.0.0-1+cuda10.0 python-libnvinfer=7.0.0-1+cuda10.0 python3-libnvinfer=7.0.0-1+cuda10.0
!sudo apt-mark hold libnvinfer7 libnvonnxparsers7 libnvparsers7 libnvinfer-plugin7 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python-libnvinfer python3-libnvinfer
!pip install pycuda 
!pip install onnx
!sudo apt-get install tensorrt

In [ ]:
!dpkg -l | grep TensorRT

In [ ]:
## installing dependencies
%%shell
git clone https://github.com/pushkar-khetrapal/RealTimePanoptic-TensorRT.git
wget https://wallup.net/wp-content/uploads/2016/01/256862-car-landscape-vehicle-road.jpg
wget https://tri-ml-public.s3.amazonaws.com/github/realtime_panoptic/models/cvpr_realtime_pano_cityscapes_standalone_no_prefix.pth
pip install yacs
pip uninstall -y apex
git clone https://www.github.com/nvidia/apex
cd apex
python setup.py install
mv '/content/RealTimePanoptic-TensorRT/realtime_panoptic' '/usr/local/lib/python3.6/dist-packages'
## attach the link of image here
wget 'https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_272722%2Fproject_255779%2Fimages%2Ftraining_0.jpg' 

In [ ]:
''' The model contains :

"Making onnx model for"(
Backone
panoptic head
)
and returns (locations, box_cls, box_regression, centerness, levelness_logits, semantic_logits, interpolated_semantic_logits)


Further these (locations, box_cls, box_regression, centerness, levelness_logits, semantic_logits, interpolated_semantic_logits)
feed into PanopticFromDenseBox.process.

'''


In [ ]:
## importing libraries
import warnings
import argparse
import torch
import numpy as np
from PIL import Image
from torchvision.models.detection.image_list import ImageList
import time

from realtime_panoptic.models.rt_pano_net import RTPanoNet
from realtime_panoptic.config import cfg
import realtime_panoptic.data.panoptic_transform as P
from realtime_panoptic.utils.visualization import visualize_segmentation_image, visualize_detection_image
import matplotlib.pyplot as plt

cityscapes_colormap = np.array([
 [128,  64, 128],
 [244,  35, 232],
 [ 70,  70,  70],
 [102, 102, 156],
 [190, 153, 153],
 [153, 153, 153],
 [250 ,170,  30],
 [220, 220,   0],
 [107, 142,  35],
 [152, 251, 152],
 [ 70, 130, 180],
 [220,  20,  60],
 [255,   0,   0],
 [  0,   0, 142],
 [  0,   0,  70],
 [  0,  60, 100],
 [  0,  80, 100],
 [  0,   0, 230],
 [119,  11,  32],
 [  0,   0,   0]])

cityscapes_instance_label_name = ['person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle']
warnings.filterwarnings("ignore", category=UserWarning)
cfg.merge_from_file('RealTimePanoptic-TensorRT/configs/demo_config.yaml')

## developing model
model = RTPanoNet(
    backbone=cfg.model.backbone, 
    num_classes=cfg.model.panoptic.num_classes,
    things_num_classes=cfg.model.panoptic.num_thing_classes,
    pre_nms_thresh=cfg.model.panoptic.pre_nms_thresh,
    pre_nms_top_n=cfg.model.panoptic.pre_nms_top_n,
    nms_thresh=cfg.model.panoptic.nms_thresh,
    fpn_post_nms_top_n=cfg.model.panoptic.fpn_post_nms_top_n,
    instance_id_range=cfg.model.panoptic.instance_id_range)
device = 'cuda'
model.to(device)

model.load_state_dict(torch.load('cvpr_realtime_pano_cityscapes_standalone_no_prefix.pth'))

output_names = ["output_0"] + ["output_%d" % i for i in range(1,24)]
input_names = ["input1"]

x = torch.randn(1, 3, 1024, 2048, requires_grad=True).to('cuda')
torch.onnx.export(model, x, "model.onnx", verbose=True, input_names=input_names, output_names=output_names)

In [ ]:
import tensorrt as trt 
import torch.onnx as onnx
import numpy as np 
import pycuda.autoinit
import time
import cv2 as cv
#import onnx

import pycuda.driver as cuda

model_path = "/content/model.onnx"
# onnx.checker.check_model(model_path)
# onnx.helper.printable_graph(model.graph)

input_size = 224

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)       
network_flags = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
def build_engine(model_path):
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network(flags = network_flags) as network, \
    trt.OnnxParser(network, TRT_LOGGER) as parser:
        builder.max_workspace_size = 1<<int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
        builder.max_batch_size = 1
        builder.fp16_mode = 1

        
        with open(model_path, 'rb') as f:
            value = parser.parse(f.read())
            print("Parser: ", value)
            if (value == False):
                print(trt.ParserError().code())
            
        engine = builder.build_cuda_engine(network)
        print(engine)
        return engine

def alloc_buf(engine):
    h_in_size = trt.volume(engine.get_binding_shape(0))
    h_out_size = trt.volume(engine.get_binding_shape(1))
    h_in_dtype = trt.nptype(engine.get_binding_dtype(0))
    h_out_dtype = trt.nptype(engine.get_binding_dtype(1))
    h_in_dtype = trt.nptype(engine.get_binding_dtype(0))
    h_out_dtype = trt.nptype(engine.get_binding_dtype(1))
    in_cpu = cuda.pagelocked_empty(h_in_size, h_in_dtype)
    out_cpu = cuda.pagelocked_empty(h_out_size, h_out_dtype)
    in_gpu = cuda.mem_alloc(in_cpu.nbytes)
    out_gpu = cuda.mem_alloc(out_cpu.nbytes)
    stream = cuda.Stream()
    return in_cpu, out_cpu, in_gpu, out_gpu, stream

def inference(engine, context, inputs, out_cpu, in_gpu, out_gpu, stream):
    cuda.memcpy_htod(in_gpu, inputs)
    context.execute(bindings=[int(in_gpu), int(out_gpu)])
    cuda.memcpy_dtoh(out_cpu, out_gpu)
    return out_cpu

if __name__ == "__main__":
    inputs = np.random.random((1, 3, 1024, 2048)).astype(np.float32)
    engine = build_engine(model_path)
    print(engine)
    context = engine.create_execution_context()
    with open("AlexNet.engine", "wb") as f:
        f.write(engine.serialize())
    for _ in range(10):
        t1 = time.time()
        in_cpu, out_cpu, in_gpu, out_gpu, stream = alloc_buf(engine)
        print(inputs.shape)
        res = inference(engine, context, inputs.reshape(-1), out_cpu, in_gpu, out_gpu, stream)
    print(len(res))
    print("cost time: ", time.time()-t1)
